In [1]:
# import required libraries

import os
import supabase
import nomic
import pandas as pd
from dotenv import load_dotenv
from nomic import atlas
from nomic import AtlasDataset
import time
import datetime


In [9]:
def data_prep_for_doc_map(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares document metadata for Nomic map upload.
    
    Args:
        df (pd.DataFrame): Source documents dataframe
        
    Returns:
        pd.DataFrame: Processed metadata for map creation, or None if error occurs
    """
    try:
      metadata = []
      current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

      for _, row in df.iterrows():
        created_at = datetime.datetime.strptime(row['created_at'],
                                                "%Y-%m-%dT%H:%M:%S.%f%z").strftime("%Y-%m-%d %H:%M:%S")

        for idx, context in enumerate(row['contexts'], 1):
          metadata.append({
              "id": f"{row['id']}_{idx}",
              "created_at": created_at,
              "s3_path": row['s3_path'],
              "url": row['url'] or "",
              "base_url": row['base_url'] or "",
              "readable_filename": row['readable_filename'],
              "modified_at": current_time,
              "text": context['text']
          })

      return pd.DataFrame(metadata)

    except Exception as e:
      print(f"Error in document data preparation: {e}")
      return pd.DataFrame()

In [10]:
def create_map(metadata, map_name, index_name, index_field):
    """
    Creates a Nomic map with topic modeling and duplicate detection.
    
    Args:
        metadata (pd.DataFrame): Metadata for the map
        map_name (str): Name of the map to create
        index_name (str): Name of the index to create
        index_field (str): Field to be indexed
        
    Returns:
        str: 'success' or error message
    """
    print(f"Creating map: {map_name}")

    try:
        project = atlas.map_data(data=metadata,
                                identifier=map_name,
                                id_field="id",
                                topic_model=True,
                                duplicate_detection=True,
                                indexed_field=index_field)

        project.create_index(name=index_name, indexed_field=index_field, topic_model=True, duplicate_detection=True)

        return "success"

    except Exception as e:
      print(e)
      return f"Error in creating map: {e}"
    
def append_to_map(metadata, map_name):
    """
    Appends new data to an existing Nomic map.
    
    Args:
        metadata (pd.DataFrame): Metadata for the map update
        map_name (str): Name of the target map
        
    Returns:
        str: 'success' or error message
    """
    try:
      print(f"Appending to map: {map_name}")
      project = AtlasDataset(map_name)

      # start_time = time.monotonic()
      # while time.monotonic() - start_time < 60:
      #   if project.is_accepting_data:
      #     project.add_data(data=metadata)
      #     print(f"Data appended to map: {map_name}")
      #     return "success"
      #   print("Project is currently indexing. Waiting for 5 seconds...")
      #   time.sleep(5)

      with project.wait_for_dataset_lock():
        project.add_data(data=metadata)
        print(f"Data appended to map: {map_name}")
        return "success"
      
      return "Project busy"

    except Exception as e:
      print(e)
      return f"Error in appending to map: {e}"

In [15]:
# read and load data into pandas df
DOCUMENT_MAP_PREFIX = "Test Document Map - "
course_name = "canvas-integration"

with open("canvas-integration.json", "r") as json_file:
	data = json.load(json_file)

df = pd.DataFrame(data)
prepared_data = data_prep_for_doc_map(df)

prepared_data

id           created_at  \
0    622357_1  2024-09-10 19:58:10   
1    622358_1  2024-09-10 19:58:10   
2    622359_1  2024-09-10 19:58:10   
3    622360_1  2024-09-10 19:58:10   
4    622361_1  2024-09-10 19:58:11   
..        ...                  ...   
250  622442_3  2024-09-10 19:58:31   
251  622442_4  2024-09-10 19:58:31   
252  622442_5  2024-09-10 19:58:31   
253  622442_6  2024-09-10 19:58:31   
254  622442_7  2024-09-10 19:58:31   

                                               s3_path url  \
0    courses/canvas-integration/5f8d932a-201c-45d9-...       
1    courses/canvas-integration/1d7e5afd-bc2c-46f5-...       
2    courses/canvas-integration/497d90a0-f045-4093-...       
3    courses/canvas-integration/10b178b3-e7a7-4649-...       
4    courses/canvas-integration/c43a80ef-29c4-4ca7-...       
..                                                 ...  ..   
250  courses/canvas-integration/c9b84629-47cf-4916-...       
251  courses/canvas-integration/c9b84629-47cf-4916-...       
252  courses/canvas-integration/c9b84629-47cf-4916-...       
253  courses/canvas-integration/c9b84629-47cf-4916-...       
254  courses/canvas-integration/c9b84629-47cf-4916-...       

                                      base_url  \
0    https://canvas.illinois.edu/courses/47206   
1    https://canvas.illinois.edu/courses/47206   
2    https://canvas.illinois.edu/courses/47206   
3    https://canvas.illinois.edu/courses/47206   
4    https://canvas.illinois.edu/courses/47206   
..                                         ...   
250  https://canvas.illinois.edu/courses/47206   
251  https://canvas.illinois.edu/courses/47206   
252  https://canvas.illinois.edu/courses/47206   
253  https://canvas.illinois.edu/courses/47206   
254  https://canvas.illinois.edu/courses/47206   

                                readable_filename          modified_at  \
0            Module-1-Page-about-your-instructors  2025-01-13 23:12:30   
1           Module-1-Page-gies-learning-platforms  2025-01-13 23:12:30   
2    Module-1-Page-live-sessions-and-office-hours  2025-01-13 23:12:30   
3                Module-1-Page-course-orientation  2025-01-13 23:12:30   
4       Module-1-Page-navigating-this-canvas-site  2025-01-13 23:12:30   
..                                            ...                  ...   
250                       FA22-MBA543-W4-Fab5.pdf  2025-01-13 23:12:30   
251                       FA22-MBA543-W4-Fab5.pdf  2025-01-13 23:12:30   
252                       FA22-MBA543-W4-Fab5.pdf  2025-01-13 23:12:30   
253                       FA22-MBA543-W4-Fab5.pdf  2025-01-13 23:12:30   
254                       FA22-MBA543-W4-Fab5.pdf  2025-01-13 23:12:30   

                                                  text  
0    MBA 543About Your Instructors\n\n\n Professor ...  
1    Gies College of BusinessGies Learning Platform...  
2    MBA 543Live Sessions and Office Hours\n\n\n Sc...  
3    MBA 543Course Orientation\n\n\n Course Welcome...  
4    Gies College of Business Navigating This Canva...  
..                                                 ...  
250  Excedrin: The Migraine Experience\nThose aroun...  
251  The Use of AR in Mobile Retail Apps\n“Augmente...  
252  Starbucks Helps People Satisfy Their Coffee Cr...  
253  Tide’s Dash Button Changed How We Buy Home \nP...  
254  Google Glass\nGoogle announced its wearable in...  

[255 rows x 8 columns]

In [16]:
index_name = f"{course_name}_doc_index"
map_title = f"{DOCUMENT_MAP_PREFIX}{course_name}"

map_creation_status = create_map(prepared_data, map_title, index_name, index_field="text")
print("Map creation status:", map_creation_status)

Creating map: Test Document Map - canvas-integration


2025-01-13 23:12:36.066 | INFO     | nomic.dataset:_create_project:839 - Organization name: `caiincsa`
2025-01-13 23:12:36.604 | INFO     | nomic.dataset:_create_project:867 - Creating dataset `test-document-map-canvas-integration`
2025-01-13 23:12:36.905 | INFO     | nomic.atlas:map_data:140 - Uploading data to Atlas.
1it [00:00,  1.72it/s]
2025-01-13 23:12:37.496 | INFO     | nomic.dataset:_add_data:1690 - Upload succeeded.
2025-01-13 23:12:37.499 | INFO     | nomic.atlas:map_data:158 - `caiincsa/test-document-map-canvas-integration`: Data upload succeeded to dataset`
2025-01-13 23:12:38.968 | INFO     | nomic.dataset:create_index:1272 - Created map `Test Document Map - canvas-integration` in dataset `caiincsa/test-document-map-canvas-integration`: https://atlas.nomic.ai/data/caiincsa/test-document-map-canvas-integration
2025-01-13 23:12:40.864 | INFO     | nomic.dataset:create_index:1272 - Created map `canvas-integration_doc_index` in dataset `caiincsa/test-document-map-canvas-integ

Map creation status: success
